In [ ]:
import os
DATA_DIR = os.environ['RNN_HOME']

In [ ]:
raw_dir = DATA_DIR + "raw/"
preprocessed_dir = DATA_DIR + "preprocessed/"
output_dir = DATA_DIR + "output/"

In [ ]:
raw_text_file = raw_dir + 'protein-test.txt'

parser_output_file = DATA_DIR + 'train/brain_pos/greedy/128-0.08-3600-0.9-0/tagged-dev-corpus'

output_file = output_dir + "protein-predicted.txt"

Load the whole test corpus as a string

In [ ]:
total_string = ""
with open(raw_text_file, 'r') as f:
    for l in f:
        total_string += l

In [ ]:
char_index = 0

current_protein = None
proteins = []

import csv

counter = 0
    
with open(parser_output_file, "r") as f:
    tsvin = csv.reader(f, delimiter='\t')
    for cols in tsvin:
        counter += 1
        '''
        if counter > 10:
            break
        '''
        if len(cols) < 4:
            continue
        token = cols[1]
        tag = cols[3]

        char_index = total_string.index(token, char_index)
        
        if (tag == "O" or tag == "BPROTEIN") and current_protein is not None:            
            proteins.append((current_protein["start"], current_protein["end"]))
            current_protein = None
        if tag == "BPROTEIN":
            current_protein = {"start":char_index}
        if current_protein is not None:
            current_protein["end"] = char_index + len(token)
            
if current_protein is not None:
    proteins.append((current_protein["start"], current_protein["end"]))

In [ ]:
token

In [ ]:
with open(output_file, "w") as f_test:
    output_writer = csv.writer(f_test, delimiter='\t')
    output_writer.writerow(["ExampleID", "Class", "Start", "End", "Entity"])
    for index, protein in enumerate(proteins):
        cols = ["T" + str(index+1), "Protein", protein[0], protein[1], total_string[protein[0]:protein[1]]]
        output_writer.writerow(cols)